In [5]:
import pandas as pd
from download.downloaders import download

In [84]:
test = pd.read_csv('data/indicator/SL1/raw/SL1_FAOSTAT.csv')

In [97]:

def process_SL1():
    df = pd.read_csv('data/indicator/SL1/raw/SL1_FAO.M.csv').query("Element == 'Cropland nutrient flow per unit area'")
    df = df.drop(columns=['Domain', 'Domain Code', 'Area Code (FAO)', 'Element Code',
                          'Item Code', 'Year Code', 'Flag', 'Flag Description', 'Unit', 'Element'])
    
    pivoted = df.pivot(index=['Area', 'Year'], columns='Item', values='Value')

    NB = pivoted['Synthetic Fertilizers'] \
        + pivoted['Manure applied to Soils'] \
        + pivoted['Atmospheric Deposition'] \
        + pivoted['Biological Fixation'] \
        - pivoted['Crop Removal']

    NB = NB.reset_index()
    NB.columns = ['Country', 'Year', 'Value']
    NB['Value'] = abs(NB['Value'])

    
    NB[NB['Value'] < 0] = 5 - NB[NB['Value'] < 0]  # mirror
    return NB



In [98]:
test = process_SL1()

In [102]:
test.query("Country == 'France'")

,Country,Year,Value
3449,France,1961,54.9417
3450,France,1962,53.8861
3451,France,1963,59.8296
3452,France,1964,62.6989
3453,France,1965,61.9145
3454,France,1966,72.4478
3455,France,1967,76.3888
3456,France,1968,84.4785
3457,France,1969,85.0429
3458,France,1970,99.3979


In [86]:
test.query("Element == 'Cropland nutrient flow per unit area'")

,Domain Code,Domain,Area Code (FAO),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
58,ESB,Soil nutrient budget,2,Afghanistan,7276,Cropland nutrient flow per unit area,5061,Synthetic Fertilizers,1961,1961,kg/ha,0.1299,Fc,Calculated data
59,ESB,Soil nutrient budget,2,Afghanistan,7276,Cropland nutrient flow per unit area,5061,Synthetic Fertilizers,1962,1962,kg/ha,0.1289,Fc,Calculated data
60,ESB,Soil nutrient budget,2,Afghanistan,7276,Cropland nutrient flow per unit area,5061,Synthetic Fertilizers,1963,1963,kg/ha,0.1280,Fc,Calculated data
61,ESB,Soil nutrient budget,2,Afghanistan,7276,Cropland nutrient flow per unit area,5061,Synthetic Fertilizers,1964,1964,kg/ha,0.1270,Fc,Calculated data
62,ESB,Soil nutrient budget,2,Afghanistan,7276,Cropland nutrient flow per unit area,5061,Synthetic Fertilizers,1965,1965,kg/ha,0.1270,Fc,Calculated data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97991,ESB,Soil nutrient budget,181,Zimbabwe,7276,Cropland nutrient flow per unit area,5078,Biological Fixation,2014,2014,kg/ha,2.1983,Fc,Calculated data
97992,ESB,Soil nutrient budget,181,Zimbabwe,7276,Cropland nutrient flow per unit area,5078,Biological Fixation,2015,2015,kg/ha,1.5661,Fc,Calculated data
97993,ESB,Soil nutrient budget,181,Zimbabwe,7276,Cropland nutrient flow per unit area,5078,Biological Fixation,2016,2016,kg/ha,1.5854,Fc,Calculated data
97994,ESB,Soil nutrient budget,181,Zimbabwe,7276,Cropland nutrient flow per unit area,5078,Biological Fixation,2017,2017,kg/ha,1.9442,Fc,Calculated data


In [61]:
test = pd.read_csv('data/full_data/result.csv')


In [83]:
test.query("Aggregation == 'Index' and Year == 2020").sort_values(by='Value').tail(10)

,ISO,Variable,Value,Aggregation,Year,Country,Continent,UNregion,IncomeLevel,Region
102016,EST,Index,70.11,Index,2020.0,Estonia,Europe,Northern Europe,High income,Europe & Central Asia
161200,ITA,Index,70.51,Index,2020.0,Italy,Europe,Southern Europe,High income,Europe & Central Asia
187867,LTU,Index,70.92,Index,2020.0,Lithuania,Europe,Northern Europe,High income,Europe & Central Asia
288490,SVK,Index,73.49,Index,2020.0,Slovakia,Europe,Eastern Europe,High income,Europe & Central Asia
122751,DEU,Index,73.77,Index,2020.0,Germany,Europe,Western Europe,High income,Europe & Central Asia
20619,AUT,Index,73.98,Index,2020.0,Austria,Europe,Western Europe,High income,Europe & Central Asia
110911,FIN,Index,74.60,Index,2020.0,Finland,Europe,Northern Europe,High income,Europe & Central Asia
87216,DNK,Index,75.45,Index,2020.0,Denmark,Europe,Northern Europe,High income,Europe & Central Asia
85728,CZE,Index,75.78,Index,2020.0,Czech Republic,Europe,Eastern Europe,High income,Europe & Central Asia
316554,SWE,Index,78.38,Index,2020.0,Sweden,Europe,Northern Europe,High income,Europe & Central Asia


In [75]:
test.query('ISO == "FRA" and Variable == "GJ1"')

,ISO,Variable,Value,Aggregation,Year,Country,Continent,UNregion,IncomeLevel,Region
111027,FRA,GJ1,0.09,Indicator,2005.0,France,Europe,Western Europe,High income,Europe & Central Asia
111063,FRA,GJ1,62.67,Indicator_normed,2005.0,France,Europe,Western Europe,High income,Europe & Central Asia
111120,FRA,GJ1,0.09,Indicator,2006.0,France,Europe,Western Europe,High income,Europe & Central Asia
111156,FRA,GJ1,62.63,Indicator_normed,2006.0,France,Europe,Western Europe,High income,Europe & Central Asia
111213,FRA,GJ1,0.09,Indicator,2007.0,France,Europe,Western Europe,High income,Europe & Central Asia
111249,FRA,GJ1,65.21,Indicator_normed,2007.0,France,Europe,Western Europe,High income,Europe & Central Asia
111306,FRA,GJ1,0.09,Indicator,2008.0,France,Europe,Western Europe,High income,Europe & Central Asia
111342,FRA,GJ1,61.74,Indicator_normed,2008.0,France,Europe,Western Europe,High income,Europe & Central Asia
111399,FRA,GJ1,0.08,Indicator,2009.0,France,Europe,Western Europe,High income,Europe & Central Asia
111435,FRA,GJ1,57.19,Indicator_normed,2009.0,France,Europe,Western Europe,High income,Europe & Central Asia


In [ ]:
test.query('ISO == "FRA" and Variable == "GN1" and Aggregation =="Indicator_normed"')

In [141]:
test = pd.read_csv('data/indicator/EQ3/preprocessed/EQ3_origin.M.csv')

In [132]:
def process_EQ3():
    df = pd.read_csv("data/indicator/EQ3/raw/EQ3_WB.csv")

    df = df[[
        'iso3c', 'country_name', 'total_msw_total_msw_generated_tons_year',
        'population_population_number_of_people'
    ]]

    df['Year'] = 2018

    df['Value'] = df['total_msw_total_msw_generated_tons_year'] / df[
        'population_population_number_of_people']

    df = df.drop(columns=[
        'total_msw_total_msw_generated_tons_year',
        'population_population_number_of_people'
    ]).rename(columns={
        'iso3c': 'ISO',
        'country_name': 'Country',
    })

    return df


In [133]:
test = process_EQ3()

In [139]:
test.query('ISO == "DEU"')

,ISO,Country,Year,Value
50,DEU,Germany,2018,0.609


In [125]:
test.query('ISO == "FRA"')

,ISO,Country,Year,Value
64,FRA,France,2018,0.548


In [118]:
test

,iso3c,region_id,country_name,income_id,gdp,composition_food_organic_waste_percent,composition_glass_percent,composition_metal_percent,composition_other_percent,composition_paper_cardboard_percent,...,waste_treatment_controlled_landfill_percent,waste_treatment_incineration_percent,waste_treatment_landfill_unspecified_percent,waste_treatment_open_dump_percent,waste_treatment_other_percent,waste_treatment_recycling_percent,waste_treatment_sanitary_landfill_landfill_gas_system_percent,waste_treatment_unaccounted_for_percent,waste_treatment_waterways_marine_percent,where_where_is_this_data_measured
0,ABW,LCN,Aruba,HIC,35563.312500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.0,NaN,89.0,NaN,NaN
1,AFG,SAS,Afghanistan,LIC,2057.062256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
2,AGO,SSF,Angola,LMC,8036.690430,51.800000,6.700000,4.400000,11.500000,11.900000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,ECS,Albania,UMC,13724.058594,51.400000,4.500000,4.800000,15.210000,9.900000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Some disposal sites
4,AND,ECS,Andorra,HIC,43711.800781,31.200000,8.200000,2.600000,11.600000,35.100000,...,NaN,52.1,NaN,NaN,NaN,NaN,NaN,47.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,XKX,ECS,Kosovo,LMC,9723.561523,42.000000,6.000000,6.000000,20.000000,8.000000,...,66.43,NaN,NaN,33.57,NaN,NaN,NaN,NaN,NaN,NaN
213,YEM,MEA,"Yemen, Rep.",LIC,8269.671875,65.000000,1.000000,6.000000,6.000000,7.000000,...,12.00,NaN,NaN,25.00,NaN,8.0,NaN,47.0,8.0,Other
214,ZAF,SSF,South Africa,UMC,12666.607422,16.381655,5.200216,16.910461,45.020646,9.396918,...,72.00,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN
215,ZMB,SSF,Zambia,LMC,3201.289307,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
test = pd.read_csv('data/indicator/SL2/preprocessed/SL2_origin.M.csv')

In [116]:
test.query("ISO == 'DEU'")

,ISO,Year,Value,Variable,Description,Source,URL,From
210,DEU,2004,4.513607,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
211,DEU,2005,4.740767,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
212,DEU,2006,4.871356,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
213,DEU,2007,5.105015,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
214,DEU,2008,5.364931,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
215,DEU,2009,5.608788,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
216,DEU,2010,5.932335,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
217,DEU,2011,6.074526,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
218,DEU,2012,6.204993,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL
219,DEU,2013,6.258609,SL2,Share agriculture organic to total agriculture...,FAOSTAT,http://www.fao.org/faostat/en/#data/EL,MANUAL


In [108]:
def process_SL2():
    df = (
        pd.read_csv('data/indicator/SL2/raw/SL2_FAOSTAT.csv')[['Year', 'Area', 'Item', 'Value']]
          
    )
    agri_orga = df.query('Item == "Agriculture area under organic agric."').set_index(["Year", 'Area'])[['Value']]
    agri_tot = df.query('Item == "Agricultural land"').set_index(['Year', 'Area'])[['Value']]
    
    df = (agri_orga / agri_tot * 100).reset_index().dropna().rename(columns={'Area': 'Country'})
    
    
    return df

In [109]:
test = process_SL2()

In [111]:
test

,Year,Country,Value
8969,2004,Algeria,0.003403
8974,2004,Argentina,1.969556
8977,2004,Australia,3.058196
8978,2004,Austria,16.070308
8985,2004,Belgium,1.710630
...,...,...,...
12361,2018,Uzbekistan,0.003687
12362,2018,Vanuatu,13.715615
12364,2018,Viet Nam,1.953299
12368,2018,Zambia,0.005160


In [110]:
test.query('Country == "Germany"')

,Year,Country,Value
9046,2004,Germany,4.513607
9272,2005,Germany,4.740767
9498,2006,Germany,4.871356
9725,2007,Germany,5.105015
9952,2008,Germany,5.364931
10179,2009,Germany,5.608788
10406,2010,Germany,5.932335
10633,2011,Germany,6.074526
10860,2012,Germany,6.204993
11087,2013,Germany,6.258609


In [59]:
df.Item.unique()

array(['Country area', 'Land area', 'Agriculture', 'Agricultural land',
       'Cropland', 'Arable land', 'Land under temporary crops',
       'Land with temporary fallow', 'Land under permanent crops',
       'Land under perm. meadows and pastures',
       'Perm. meadows & pastures - Nat. growing', 'Forest land',
       'Naturally regenerating forest', 'Planted Forest', 'Other land',
       'Land area equipped for irrigation',
       'Land area actually irrigated',
       'Agriculture area actually irrigated',
       'Agriculture area under organic agric.',
       'Agriculture area certified organic', 'Primary Forest',
       'Land under temp. meadows and pastures',
       'Land under protective cover', 'Inland waters', 'Coastal waters',
       'Cropland area actually irrigated',
       'Cropland area under organic agric.',
       'Cropland area certified organic',
       'Perm. meadows & pastures - Cultivated',
       'Land used for aquaculture',
       'Perm. meadows & pastures area

In [32]:
def process_SL1():

    df = pd.read_csv('data/indicator/SL1/raw/SL1_FAO.M.csv')
    df = df.rename(columns={'Area': 'Country'})
    # df['Value'] = abs(df['Value'])
    df[df['Value'] < 0] = 5 - df[df['Value'] < 0]  # mirror
    return df[['Country', 'Year', 'Value']]

In [33]:
test = process_SL1()

In [42]:
def process_SL1():

    df = pd.read_csv('data/indicator/SL1/raw/SL1_FAOSTAT.csv')
    df = df.rename(columns={'Area': 'Country'})
    df['Value'] = abs(df['Value'])
    #df[df['Value'] < 0] = 5 - df[df['Value'] < 0]  # mirror
    return df#[['Country', 'Year', 'Value']]

In [43]:
test = process_SL1()

In [56]:
test.query('Country == "France" and Element == "Cropland nutrient flow per unit area"').groupby('Year').Value.sum()

Year
1961    106.0743
1962    116.3869
1963    120.2054
1964    126.5649
1965    132.1659
1966    136.4552
1967    153.6798
1968    165.6153
1969    164.3615
1970    175.7727
1971    188.9591
1972    196.6313
1973    214.1624
1974    198.0024
1975    200.5307
1976    205.9426
1977    212.0197
1978    225.1724
1979    234.1446
1980    240.8038
1981    241.3355
1982    244.5665
1983    248.4831
1984    262.0845
1985    262.8445
1986    266.2662
1987    269.3108
1988    281.4847
1989    288.3010
1990    282.1630
1991    286.1787
1992    263.1035
1993    260.1236
1994    262.9645
1995    264.0393
1996    278.8200
1997    280.0844
1998    284.1226
1999    281.8115
2000    268.0245
2001    264.9279
2002    263.9578
2003    256.7703
2004    268.1915
2005    254.0736
2006    249.8770
2007    256.1299
2008    251.1474
2009    242.2532
2010    247.3833
2011    237.2719
2012    243.9458
2013    250.6901
2014    254.7463
2015    260.1912
2016    239.4474
2017    257.0017
2018    254.0916
Name: Val

In [23]:
#pd.read_csv('data/indicator/ME2/preprocessed/ME2_origin.M.csv')

In [24]:
test = process_ME2()

In [25]:
test.query('Country == "France"')

,Country,Description,Year,Value
3719,France,MF tonnes per capita,1990,17.034240
3902,France,MF tonnes per capita,1991,17.360431
4085,France,MF tonnes per capita,1992,19.902707
4268,France,MF tonnes per capita,1993,18.749216
4451,France,MF tonnes per capita,1994,18.855029
4634,France,MF tonnes per capita,1995,21.092498
4817,France,MF tonnes per capita,1996,19.065489
5000,France,MF tonnes per capita,1997,17.789807
5183,France,MF tonnes per capita,1998,19.197151
5366,France,MF tonnes per capita,1999,19.541595


In [6]:
test.query('Country == "Germany"')

,Country,Flow Type,1970,1971,1972,1973,1974,1975,1976,1977,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
63,Germany,MF tonnes per capita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.021508,21.285698,20.910101,21.048596,21.551255,21.880811,22.220808,22.586434,NaN,NaN


In [13]:
import pandas as pd
from processing.utils import add_ISO

def process_GN1():
    df = (
        pd.read_csv('data/indicator/GN1/raw/GN1_WIPO.M.csv')
          .drop(columns=['Origin', 'Office (Code)']).rename(columns={'Office': 'Country'})
          .melt(id_vars=['Country', 'Field of technology'], var_name='Year', value_name='Value')
    )
    
    df = add_ISO(df)
    
    env = df[df['Field of technology'] == '24 - Environmental technology'].drop(columns=['Field of technology']).set_index(['Country', 'ISO', 'Year'])
    total = df.groupby(['Country', 'ISO', 'Year']).sum()
    
    df = (env / total).reset_index()
    df['Value'] = df.groupby(['ISO'])['Value'].transform(lambda x: x.rolling(7, 1).mean())

    return df

In [15]:
df = process_GN1()
df.query('ISO == "FRA"')

,Country,ISO,Year,Value
1120,France,FRA,1980,0.014346
1121,France,FRA,1981,0.013340
1122,France,FRA,1982,0.012924
1123,France,FRA,1983,0.012545
1124,France,FRA,1984,0.012072
1125,France,FRA,1985,0.012032
1126,France,FRA,1986,0.011996
1127,France,FRA,1987,0.011627
1128,France,FRA,1988,0.011449
1129,France,FRA,1989,0.011587


In [ ]:
df = process_GN1()

In [ ]:
df.query('ISO == "DEU"').plot(kind='line', x='Year', y='Value')

In [ ]:
df

In [ ]:
df = pd.read_csv('data/indicator/GE3/preprocessed/GE3.1_CW.csv')

In [ ]:
test = pd.read_csv('data/full_data/result.csv')

In [ ]:
test.query("Year == 2020 and Aggregation == 'Index'")

In [ ]:
import json
with open('data/indicator/GE3/raw/GE3.1_CW.json', 'r') as outfile:
    data = json.load(outfile)


In [ ]:
config_3 = {
            "GGI_code": "GE3.1",
            "params": {
                "gas_ids[]": [
                    327,
                    328,
                    329
                ],
                "sector_ids[]": [
                    1304,
                    1302
                ],
                "source_ids[]": [111]
            }
        }

In [ ]:
data_json = download('CW', config_3, path=None, restart=False)

In [ ]:
data_json

In [ ]:
from processing.api_preprocessors import CW_Preprocessor, preprocess_raw_dict

In [ ]:
df = preprocess_raw_dict(data_json, CW_Preprocessor)

In [ ]:
df

In [ ]:
test = pd.json_normalize(data_json['data'])

In [ ]:
test

In [ ]:
import requests

def get_CW_API_ids(query_param, API_URL='https://www.climatewatchdata.org/'):
    """

    """            
    url     = f'{API_URL}/api/v1/data/historical_emissions/{query_param}'

    response = requests.get(url, params=query_param, headers={"Accept": "application/json"})
    
    
    data_json = response.json()
    df = pd.json_normalize(data_json['data']) 
    
    return df

In [ ]:
test = get_CW_API_ids(query_param="sectors")


In [ ]:
test

In [ ]:
gases